In [6]:
# Load data
import pandas as pd 

file_path='assets/datasets/clean/250604-consolidated_survey.xlsx'
df_load=pd.read_excel(file_path).drop(columns=['Unnamed: 0'])

In [7]:
df_load.head()

,start_date,end_date,survey_status,completion_level,completion_time,survey_completed,recorded_date,response_id,distribution_channel,user_language,...,soc_aspects,soc_aspects_other,es_assess,es_assess_other,es_more_info,es_certif,email,survey,inv_platform,inv_platform_other
0,2024-11-13 10:30:58,2024-11-13 10:37:24,0,100,386,1,2024-11-13 10:37:25.745,R_2CYvoda8TZhYFaN,anonymous,IT,...,"2,3,4",NaN,2,NaN,3.0,"2,3",NaN,BA,NaN,NaN
1,2024-11-14 14:00:16,2024-11-14 14:08:08,0,100,472,1,2024-11-14 14:08:09.160,R_2PCMoZEjgiOeupr,anonymous,IT,...,1,NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
2,2024-11-14 14:28:17,2024-11-14 14:36:18,0,100,480,1,2024-11-14 14:36:18.958,R_8HBDPZSIjz1M0rn,anonymous,IT,...,"1,3",NaN,3,NaN,5.0,"2,3",anmata@libero.it,BA,NaN,NaN
3,2024-11-14 16:46:00,2024-11-14 18:31:25,0,100,6324,1,2024-11-14 18:31:26.274,R_8dSEqbK9X2kHC6i,anonymous,IT,...,"1,4",NaN,2,NaN,3.0,1,NaN,BA,NaN,NaN
4,2024-11-14 18:26:48,2024-11-14 18:42:58,0,100,969,1,2024-11-14 18:42:58.484,R_2kq5hZdQfhR6sbI,anonymous,IT,...,"2,3",NaN,2,NaN,4.0,1,francoagostinoperone@gmail.com,BA,NaN,NaN


In [8]:
df_load.columns

Index(['start_date', 'end_date', 'survey_status', 'completion_level',
       'completion_time', 'survey_completed', 'recorded_date', 'response_id',
       'distribution_channel', 'user_language', 'recaptcha_score',
       'gender_male', 'age', 'children_num', 'edu_level', 'edu_field',
       'edu_field_other', 'working_status', 'working_status_other',
       'expertise_area', 'expertise_area_other', 'work_industries',
       'work_industries_other', 'work_company_size', 'roles_cat', 'inv_time',
       'inv_motivation', 'inv_motivation_other', 'ba_group', 'ba_group_name',
       'ba_group_name_other', 'ba_group_reason', 'ba_group_reason_other',
       'inv_num', 'inv_companies_last2y', 'inv_amount_average',
       'inv_capital_stake', 'inv_dd_duration', 'inv_criteria',
       'inv_criteria_other', 'inv_sectors', 'inv_sectors_other', 'inv_process',
       'inv_process_other', 'inv_visits', 'non_fin_contr',
       'non_fin_contr_other', 'child_bias', 'inv_goal', 'inv_exit',
       'inv_re

In [ ]:
# Proportions of women and men in both surveys

pd.options.display.max_columns = None
columns={
    'gender_male': 'gender',
    'start_date': 'percentage',
}
# df_temp=df_load[['survey', 'gender_male', 'age', 'children_num', 'edu_level']].groupby(['survey']).mean()
df_temp=df_load.copy()
df_temp['gender_male']=df_temp['gender_male'].replace(1, 'women')
df_temp['gender_male']=df_temp['gender_male'].replace(2, 'men')
df_temp=df_temp.dropna(subset=['gender_male'])
df_stats=(df_temp.groupby(['survey', 'gender_male'])['start_date'].count()).div(df_temp.groupby(['survey'])['start_date'].count())
df_stats=df_stats.reset_index().rename(columns=columns)
df_stats

,survey,gender,percentage
0,BA,men,0.712871
1,BA,women,0.287129
2,CI,men,0.617021
3,CI,women,0.382979
